1. El Problema: Predicción de Precios de Casas
Vamos a simular un problema clásico de regresión. Tenemos un DataFrame con 3 columnas (Características o Features):

* Metros Cuadrados (ej: 50 a 200).

* Antigüedad (ej: 1 a 50 años).

* Distancia al centro (ej: 0.5 a 20 km).

Y queremos predecir el Precio (Target).

2. El Puente: De Pandas a PyTorch
Aquí hay una regla de oro que un Senior Engineer nunca olvida:

⚠️ Regla de Oro: A las redes neuronales NO les gustan los números grandes ni los rangos dispares.

Si metes "100 metros" y "0.5 km" juntos, la red se confundirá porque 100 es mucho más grande que 0.5, aunque la distancia sea importante. Solución: Debemos normalizar los datos (hacer que todo esté más o menos entre -1 y 1) antes de convertirlos a Tensores.

Usaremos StandardScaler de Scikit-Learn para esto.

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 1. Simulación de un CSV cargado en Pandas
data = {
    'metros': np.random.randint(50, 200, 1000),
    'antiguedad': np.random.randint(1, 50, 1000),
    'distancia': np.random.uniform(0.5, 20, 1000),
    'precio': np.zeros(1000) # Placeholder
}
df = pd.DataFrame(data)

# Generamos un precio lógico: Precio base + (Metros * 1000) - (Antigüedad * 500) ...
df['precio'] = 50000 + (df['metros'] * 1000) - (df['antiguedad'] * 500) - (df['distancia'] * 1000)

# 2. Separamos Features (X) y Target (y)
X_numpy = df[['metros', 'antiguedad', 'distancia']].values
y_numpy = df['precio'].values.reshape(-1, 1) # Importante: reshape a columna

# 3. NORMALIZACIÓN (CRÍTICO PARA REDES NEURONALES)
scaler_x = StandardScaler()
scaler_y = StandardScaler() # Escalamos también el precio para que la Loss no sea gigante

X_scaled = scaler_x.fit_transform(X_numpy)
y_scaled = scaler_y.fit_transform(y_numpy)

print(f"Datos listos.")
print(f"Ejemplo X normalizado: {X_scaled[0]}")
print(f"Ejemplo y normalizado: {y_scaled[0]}")

Datos listos.
Ejemplo X normalizado: [ 1.64854767 -0.33145076  1.49200015]
Ejemplo y normalizado: [1.4850106]


Tu Misión:

1) Crea la clase HouseDataset:

* En el __init__, debe recibir dos argumentos: features y targets.

* Dentro del __init__, convierte esos arrays de numpy a Tensores de PyTorch tipo float32.

* Pista: torch.tensor(..., dtype=torch.float32)

Implementa __len__ y __getitem__.

2) Crea la clase HouseModel:

* Entrada: 3 neuronas (porque tenemos 3 features).

* Oculta: 2 capas ocultas de 64 neuronas cada una con ReLU.

* Salida: 1 neurona (el precio). ¡OJO! No uses activación en la salida final (queremos un valor lineal continuo, no una probabilidad).

Escribe el código de estas dos clases y instáncialas.

Instancia el dataset pasando X_scaled y y_scaled.

Instancia el modelo.

In [2]:
class HouseDataset(Dataset):
    def __init__(self, features, targets):
        self.features=torch.tensor(data=features, dtype=torch.float32)
        self.targets=torch.tensor(data=targets, dtype=torch.float32)
    
    def __len__(self) -> int:
        return len(self.features)
    
    def __getitem__(self, index):
        # DEVOLVEMOS TUPLA: (Datos, Precio)
        return self.features[index], self.targets[index]

class HouseModel(nn.Module):
    def __init__(self):
        super().__init__()
        #primera de 3 a 64 ocultas. Entrada -> Oculta
        self.capa1=nn.Linear(in_features=3,out_features=64)
        self.activacion=nn.ReLU()
        #oculta -> oculta. segunda transformación (Profundidad): De 64 a 64
        self.capa2=nn.Linear(in_features=64,out_features=64)
        #capa Final (Compresión): De 64 a 1 solo precio
        self.salida=nn.Linear(in_features=64,out_features=1)
    
    def forward(self, x):
        """Debe pasar x a través de: capa1 -> activacion -> capa2 -> activacion -> salida."""
        x=self.capa1(x)
        x=self.activacion(x)
        x=self.capa2(x)
        x=self.activacion(x)
        x=self.salida(x)
        return x

modelo=HouseModel()
dataset=HouseDataset(features=X_scaled, targets=y_scaled)

Vas a escribir el script de entrenamiento completo tú solo, uniendo todo lo que hemos visto.

Requisitos Técnicos:

* DataLoader: Instáncialo con tu dataset, batch_size=32 y shuffle=True.

* Loss Function: Como es un problema de regresión (predecir valores continuos), DEBES usar nn.MSELoss() (Error Cuadrático Medio).

* Optimizador: Usa torch.optim.Adam con un learning rate (lr) de 0.001. Adam suele funcionar mejor que SGD para estos datos.

* Bucle: Entrena por 20 épocas.

In [3]:
#Crear el DataLoader
#batch_size=32: Entregará paquetes de 32 datos
#shuffle=True: Barajará los datos en cada época
loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True)

#loss function
loss_function=nn.MSELoss()
#optimizador
optimizador=torch.optim.Adam(params=modelo.parameters(),lr=0.001)
#bucle
epochs=20

for epoch in range(epochs):
    loss_acum = 0
    
    for batch_x, batch_y in loader:      
        # 1. Forward
        prediccion = modelo(batch_x)
        
        # 2. Calcular Loss
        loss = loss_function(prediccion, batch_y)
        
        # 3. Limpiar gradientes previos
        optimizador.zero_grad()
        
        # 4. Backward (Calcular gradientes)
        loss.backward()
        
        # 5. Actualizar pesos
        optimizador.step()
        
        # Acumulamos el error para monitorear (opcional)
        loss_acum += loss.item()
    
    # Reporte por epoch
    promedio_loss = loss_acum / len(loader)
    print(f"Época {epoch+1} | Loss Promedio: {promedio_loss:.5f}")

Época 1 | Loss Promedio: 0.42114
Época 2 | Loss Promedio: 0.01153
Época 3 | Loss Promedio: 0.00381
Época 4 | Loss Promedio: 0.00235
Época 5 | Loss Promedio: 0.00204
Época 6 | Loss Promedio: 0.00156
Época 7 | Loss Promedio: 0.00121
Época 8 | Loss Promedio: 0.00100
Época 9 | Loss Promedio: 0.00079
Época 10 | Loss Promedio: 0.00065
Época 11 | Loss Promedio: 0.00052
Época 12 | Loss Promedio: 0.00042
Época 13 | Loss Promedio: 0.00036
Época 14 | Loss Promedio: 0.00032
Época 15 | Loss Promedio: 0.00026
Época 16 | Loss Promedio: 0.00022
Época 17 | Loss Promedio: 0.00020
Época 18 | Loss Promedio: 0.00017
Época 19 | Loss Promedio: 0.00015
Época 20 | Loss Promedio: 0.00015


Llega un cliente a tu inmobiliaria.

La Casa:Metros: 120 $m^2$

Antigüedad: 10 años

Distancia: 5.0 km

Tu Misión:Usa tu modelo entrenado para predecir el precio de esta casa.

In [4]:
# 1. Definimos los datos de la casa nueva (en el mismo orden que entrenamos)
# [metros, antiguedad, distancia]
casa_nueva = np.array([[120, 10, 5.0]]) 

# 2. Normalizamos la entrada (Usamos el MISMO scaler_x del entrenamiento)
casa_nueva_scaled = scaler_x.transform(casa_nueva)

# 3. Convertimos a Tensor (recuerda el tipo de dato y el dispositivo)
# RELLENA AQUI: casa_tensor = ...
casa_tensor = torch.tensor(casa_nueva_scaled, dtype=torch.float32)

# 4. Predicción
modelo.eval() # Modo evaluación
with torch.no_grad(): # Apagamos gradientes
    # RELLENA AQUI: Haz el pase forward
    prediccion_scaled = modelo(casa_tensor)

# 5. Des-normalizamos el precio (De vuelta a dólares)
# Usamos scaler_y.inverse_transform
# OJO: inverse_transform espera un array de numpy, no un tensor.
precio_final = scaler_y.inverse_transform(prediccion_scaled.numpy())

print(f"El precio estimado es: ${precio_final[0][0]:,.2f}")

El precio estimado es: $159,905.00


In [5]:
# 1. Creamos la etiqueta binaria basada en el precio real
# Si precio > 150000 -> 1 (VIP), si no -> 0 (Normal)
umbral = 150000
y_binario = (df['precio'].values > umbral).astype(int).reshape(-1, 1)

# 2. Convertimos a Tensor Float32
# ¡OJO! BCELoss espera que las etiquetas sean Float, no Long/Int.
y_train_class = torch.tensor(y_binario, dtype=torch.float32)

# 3. Reusamos tus Features normalizados (X_scaled) que ya tenías
X_train_tensor = torch.tensor(X_scaled, dtype=torch.float32)

# 4. Creamos el nuevo Dataset y Loader
dataset_class = HouseDataset(X_train_tensor, y_train_class)
loader_class = DataLoader(dataset_class, batch_size=32, shuffle=True)

print(f"Total de casas VIP: {y_binario.sum()} de {len(y_binario)}")

Total de casas VIP: 501 de 1000


C:\Users\Dark7 120gb\AppData\Local\Temp\ipykernel_21164\4103088138.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features=torch.tensor(data=features, dtype=torch.float32)
C:\Users\Dark7 120gb\AppData\Local\Temp\ipykernel_21164\4103088138.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets=torch.tensor(data=targets, dtype=torch.float32)


Tu Misión:

Escribe el script de entrenamiento para Clasificación.

* Instancia el modelo: modelo = HouseModel() (Reinstáncialo para borrar los pesos viejos).

* Define la Loss: Usa nn.BCEWithLogitsLoss().

* Define el Optimizador: Adam, lr=0.001.Bucle: Entrena por 20 épocas.

💡 Reto Extra (Opcional pero recomendado):En el print final de cada época, además de la Loss, intenta calcular la Accuracy (Precisión).

Pista: La predicción cruda (logits) pasa por una sigmoide. Si el resultado $> 0.5$, es un 1. Compara eso con batch_y.

In [6]:
modelo=HouseModel()
loss_function=nn.BCEWithLogitsLoss()
optimizador=torch.optim.Adam(params=modelo.parameters(),lr=0.001)
epochs=20
for epoch in range(epochs):
    loss_acumulada=0
    correctos_acumulados = 0
    total_muestras = 0

    for batch_x, batch_y in loader_class:      
        # 1. Forward
        prediccion = modelo(batch_x)
        # 2. Calcular Loss
        loss = loss_function(prediccion, batch_y)
        # 3. Limpiar gradientes previos
        optimizador.zero_grad()
        # 4. Backward (Calcular gradientes)
        loss.backward()
        # 5. Actualizar pesos
        optimizador.step()
        # Acumulamos el error para monitorear (opcional)
        loss_acumulada += loss.item()

        #calculo accuracy
        #convertir logits a probabilidades
        probs = torch.sigmoid(prediccion)
        #Convertir a 0 o 1 (Decisión)
        preds = (probs > 0.5).float()
        #Contar aciertos (True si son iguales, False si no)
        correctos_acumulados += (preds == batch_y).sum().item()
        total_muestras += batch_y.size(0) # Sumamos 32 (tamaño del batch)
    # Reporte por epoch
    promedio_loss = loss_acumulada / len(loader_class)
    accuracy_epoch = correctos_acumulados / total_muestras
    print(f"Époch {epoch+1} | Loss Promedio: {promedio_loss:.5f} | Accuracy: {accuracy_epoch*100:.2f}%")

Époch 1 | Loss Promedio: 0.59347 | Accuracy: 75.30%
Époch 2 | Loss Promedio: 0.34230 | Accuracy: 94.00%
Époch 3 | Loss Promedio: 0.17185 | Accuracy: 97.90%
Époch 4 | Loss Promedio: 0.10501 | Accuracy: 98.70%
Époch 5 | Loss Promedio: 0.07580 | Accuracy: 99.10%
Époch 6 | Loss Promedio: 0.05920 | Accuracy: 99.60%
Époch 7 | Loss Promedio: 0.05098 | Accuracy: 99.60%
Époch 8 | Loss Promedio: 0.04164 | Accuracy: 99.50%
Époch 9 | Loss Promedio: 0.03654 | Accuracy: 99.60%
Époch 10 | Loss Promedio: 0.03300 | Accuracy: 99.70%
Époch 11 | Loss Promedio: 0.02990 | Accuracy: 99.80%
Époch 12 | Loss Promedio: 0.02707 | Accuracy: 99.80%
Époch 13 | Loss Promedio: 0.02477 | Accuracy: 99.80%
Époch 14 | Loss Promedio: 0.02190 | Accuracy: 99.90%
Époch 15 | Loss Promedio: 0.02073 | Accuracy: 99.90%
Époch 16 | Loss Promedio: 0.02053 | Accuracy: 99.80%
Époch 17 | Loss Promedio: 0.01785 | Accuracy: 99.90%
Époch 18 | Loss Promedio: 0.01588 | Accuracy: 99.90%
Époch 19 | Loss Promedio: 0.01584 | Accuracy: 100.00%
É

In [7]:
print("Así se ven tus parámetros entrenados:")
print(modelo.state_dict().keys())
# Verás algo como: 'capa1.weight', 'capa1.bias', etc.

Así se ven tus parámetros entrenados:
odict_keys(['capa1.weight', 'capa1.bias', 'capa2.weight', 'capa2.bias', 'salida.weight', 'salida.bias'])


In [8]:
torch.save(modelo.state_dict(), "modelo_casas_vip.pth")
print("¡Modelo guardado en el disco!")

¡Modelo guardado en el disco!


In [9]:
# --- TU CÓDIGO CORREGIDO ---
torch.save(modelo.state_dict(), "mi_mejor_modelo.pth")

nuevo_modelo = HouseModel()
checkpoint = torch.load("mi_mejor_modelo.pth") 
nuevo_modelo.load_state_dict(checkpoint)

# Corrección aquí: usamos la variable que acabas de crear
nuevo_modelo.eval() 

# --- LO QUE NO ENTENDÍAS (LA PRUEBA DE FUEGO) ---
# Usamos el tensor de la casa de prueba (120m2, 10 años...)
# Asumimos que 'casa_tensor' todavía existe en tu memoria del ejercicio anterior.
# Si no, recuerda: era la casa de 160.000 USD (que ES VIP).

with torch.no_grad():
    # 1. Pasamos la casa por el modelo NUEVO
    logits = nuevo_modelo(casa_tensor)
    
    # 2. Convertimos a probabilidad (0 a 1)
    probabilidad = torch.sigmoid(logits)

print(f"Probabilidad de ser VIP: {probabilidad.item():.4f}")
# Si el modelo aprendió bien, esto debería ser muy cercano a 1.0 (ej: 0.99)

Probabilidad de ser VIP: 0.9958
